# Advanced Querying Mongo

**⚠️ IMPORTANT: Limit your print to avoid infinite scrolling. Otherwise your
code will be lost between print lines. If working with lists do:**

```python
list(collection.find(query))[:5] #or a reasonably low number
```

Importing libraries and setting up connection

In [11]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")
db = client["Ironhack"]
client.list_database_names()
c = db.get_collection("companies")



### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [13]:
filter_ = {"name": "Babelgum"}
projection = {"name": 1, "_id": 0}

c.find_one(filter_, projection)

{'name': 'Babelgum'}

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [80]:
filter_ = {"number_of_employees": {"$gt":5000}}
projection = {"name": 1, "number_of_employees": 1, "_id": 0}

list(c.find(filter_ , projection).sort("number_of_employees", -1).limit(20))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000},
 {'name': 'Sony', 'number_of_employees': 180500},
 {'name': 'LG', 'number_of_employees': 177000},
 {'name': 'Ford', 'number_of_employees': 171000},
 {'name': 'Boeing', 'number_of_employees': 160000},
 {'name': 'Digital Equipment Corporation', 'number_of_employees': 140000},
 {'name': 'Nokia', 'number_of_employees': 125000},
 {'name': 'MItsubishi Electric', 'number_of_employees': 107000}

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [81]:
filter_ = {"founded_year": {"$gte":2000}}
projection = {"name": 1, "founded_year": 1, "_id": 0}

condition_1 = {"founded_year": {"$gte":2000}}
condition_2 = {"founded_year": {"$lte":2005}}

list(c.find({
            "$and": 
                [condition_1, condition_2]}, projection).limit(5))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [82]:
projection = {"name": 1, "ipo.valuation_amount": 1, "_id": 0}

condition_1 = {"ipo.valuation_amount": {"$gt":100000000}}
condition_2 = {"founded_year": {"$lt":2010}}

list(c.find({
            "$and": 
                [condition_1, condition_2]}, projection).limit(5))

[{'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Yelp', 'ipo': {'valuation_amount': 1300000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [83]:
filter_ = {"founded_year": {"$gte":2000}}
projection = {"name": 1, "number_of_employees": 1, "founded_year": 1, "_id": 0}

condition_1 = {"number_of_employees": {"$lt":1000}}
condition_2 = {"founded_year": {"$lt":2005}}

list(c.find({
            "$and": 
                [condition_1, condition_2]}, projection).sort("number_of_employees", -1).limit(10))

[{'name': 'Infinera Corporation',
  'number_of_employees': 974,
  'founded_year': 2000},
 {'name': 'NorthPoint Communications Group',
  'number_of_employees': 948,
  'founded_year': 1997},
 {'name': '888 Holdings', 'number_of_employees': 931, 'founded_year': 1997},
 {'name': 'Forrester Research',
  'number_of_employees': 903,
  'founded_year': 1983},
 {'name': 'SonicWALL', 'number_of_employees': 900, 'founded_year': 1991},
 {'name': 'Webmetrics', 'number_of_employees': 900, 'founded_year': 1999},
 {'name': 'Cornerstone OnDemand',
  'number_of_employees': 881,
  'founded_year': 1999},
 {'name': 'Mozilla', 'number_of_employees': 800, 'founded_year': 1998},
 {'name': 'Buongiorno', 'number_of_employees': 800, 'founded_year': 1999},
 {'name': 'Yelp', 'number_of_employees': 800, 'founded_year': 2004}]

### 6. All the companies that don't include the `partners` field.

In [84]:
filter_ = {"partners": {"$exists": False}}
projection = {"name": 1, "_id": 0}

list(c.find(filter_ , projection).limit(5))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [85]:
filter_ = {"category_code": {"$type": "null"}}
projection = {"name": 1, "_id": 0}

list(c.find(filter_ , projection).limit(5))

[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [86]:
projection = {"name": 1, "number_of_employees": 1, "_id": 0}

condition_1 = {"number_of_employees": {"$gte":100}}
condition_2 = {"number_of_employees": {"$lt":1000}}

list(c.find({
            "$and": 
                [condition_1, condition_2]}, projection).limit(5))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110}]

### 9. Order all the companies by their IPO price in a descending order.

In [87]:
filter_ = {"ipo.valuation_amount": {"$exists": True}}
projection = {"name": 1, "_id": 0}

list(c.find(filter_ , projection).sort("ipo.valuation_amount", -1).limit(5))





[{'name': 'GREE'},
 {'name': 'Facebook'},
 {'name': 'Amazon'},
 {'name': 'Twitter'},
 {'name': 'Groupon'}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [88]:
filter_ = {"number_of_employees": {"$exists": True}}
projection = {"name": 1, "_id": 0, "number_of_employees": 1}

list(c.find(filter_ , projection).sort("number_of_employees", -1).limit(10).limit(5))





[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [89]:
projection = {"name": 1, "_id": 0}

condition_1 = {"founded_month": {"$gte":6}}
condition_2 = {"founded_month": {"$lt":12}}

list(c.find({
            "$and": 
                [condition_1, condition_2]}, projection).limit(1000).limit(5))

[{'name': 'Wetpaint'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Omnidrive'},
 {'name': 'Postini'}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.000

In [90]:
projection = {"name": 1, "_id": 0}

condition_1 = {"founded_year": {"$lt":2000}}
condition_2 = {"acquisition.price_amount": {"$gt":10000000}}

list(c.find({
            "$and": 
                [condition_1, condition_2]}, projection).limit(5))

[{'name': 'Postini'},
 {'name': 'SideStep'},
 {'name': 'Recipezaar'},
 {'name': 'PayPal'},
 {'name': 'Snapfish'}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [91]:
projection = {"name": 1, "acquisition": 1, "_id": 0}

condition_1 = {"acquisition.acquired_year": {"$lt":2010}}

list(c.find({
            "$and": 
                [condition_1]}, projection).sort("acquisition.price_amount", 1).limit(5))

[{'name': 'Gannett',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.clickz.com/clickz/news/1717294/gannett-buys-pointroll',
   'source_description': 'Gannett Buys PointRoll',
   'acquired_year': 2005,
   'acquired_month': 6,
   'acquired_day': 10,
   'acquiring_company': {'name': 'PointRoll', 'permalink': 'pointroll'}}},
 {'name': 'Gizmoz',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.gizmoz.com',
   'source_description': 'TechCrunch',
   'acquired_year': 2009,
   'acquired_month': 12,
   'acquired_day': 15,
   'acquiring_company': {'name': 'Daz 3d', 'permalink': 'daz-3d'}}},
 {'name': 'Livestream',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': 'cash_and_stock',
   'source_url': '',
   'source_description': 'company',
   'acquired_year': 2008,
   'acquired_month': 1,
   'acquired_day': 1,
 

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [92]:
filter_ = {"founded_year": {"$exists": True}}
projection = {"name": 1, "_id": 0, "founded_year": 1}
list(c.find(filter_ , projection).sort("founded_year", -1).limit(10))





[{'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'Fixya', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [93]:
projection = {"name": 1, "_id": 0}

condition_1 = {"founded_day": {"$lte":7}}
condition_2 = {"founded_day": {"$gte":1}}

list(c.find({
            "$and": 
                [condition_1, condition_2]}, projection).sort("acquisition.price_amount", -1).limit(10))

[{'name': 'Netscape'},
 {'name': 'PayPal'},
 {'name': 'Zappos'},
 {'name': 'Alibaba'},
 {'name': 'Postini'},
 {'name': 'Danger'},
 {'name': 'Clearwell Systems'},
 {'name': 'PrimeSense'},
 {'name': 'Amobee'},
 {'name': 'BlueLithium'}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [94]:
projection = {"name": 1, "_id": 0, "number_of_employees":1}

condition_1 = {"number_of_employees": {"$gte":4000}}
condition_2 = {"category_code": "web"}

list(c.find({
            "$and": 
                [condition_1, condition_2]}, projection).sort("number_of_employees", 1).limit(10))

[{'name': 'Expedia', 'number_of_employees': 4400},
 {'name': 'AOL', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'number_of_employees': 8657},
 {'name': 'Rakuten', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group', 'number_of_employees': 10000},
 {'name': 'Groupon', 'number_of_employees': 10000},
 {'name': 'Yahoo!', 'number_of_employees': 13600},
 {'name': 'eBay', 'number_of_employees': 15000},
 {'name': 'Experian', 'number_of_employees': 15500}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [95]:
projection = {"name": 1, "_id": 0, }

condition_1 = {"acquisition.price_amount": {"$gt":10000000}}
condition_2 = {"acquisition.price_currency_code": "EUR"}

list(c.find({
            "$and": 
                [condition_1, condition_2]}, projection).limit(5))

[{'name': 'ZYB'},
 {'name': 'Apertio'},
 {'name': 'Greenfield Online'},
 {'name': 'Webedia'},
 {'name': 'Wayfinder'}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [96]:
projection = {"name": 1, "acquisition":1, "_id": 0}

condition_1 = {"acquisition.acquired_month": {"$gte":1}}
condition_2 = {"acquisition.acquired_month": {"$lte":3}}

list(c.find({
            "$and": 
                [condition_1, condition_2]}, projection).limit(10))

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [97]:
projection = {"name": 1, "_id": 0}

condition_1 = {"founded_year": {"$gte":2000}}
condition_2 = {"founded_year": {"$lte":2010}}
condition_3 = {"acquisition.acquired_year": {"$exists": False}} 
condition_4 = {"acquisition.acquired_year": {"$gt":2011}} 

list(c.find({
            "$and": 
                [condition_1, condition_2, {
                    "$or": [
                        condition_3,
                        condition_4
                    ]
                }
            ]}, projection).limit(10))



[{'name': 'Wetpaint'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Facebook'},
 {'name': 'Omnidrive'},
 {'name': 'Geni'},
 {'name': 'Twitter'},
 {'name': 'Scribd'},
 {'name': 'Slacker'},
 {'name': 'MeetMoi'}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [79]:
filter_ = {"deadpooled_year": {"$gte": 3}}
projection = {"name": 1, "_id": 0}
list(c.find(filter_ , projection).limit(5))





[{'name': 'Zoho'},
 {'name': 'Omnidrive'},
 {'name': 'Babelgum'},
 {'name': 'Sparter'},
 {'name': 'Thoof'}]

**⚠️ Did you do this?**

```python
list(collection.find(query))[:5] #or a reasonably low number
```